# Wikipedia記事の検索用埋め込み

このノートブックは、[Question_answering_using_embeddings.ipynb](Question_answering_using_embeddings.ipynb)で使用される、Wikipediaの記事のデータセットを検索用に準備した方法を示しています。

手順：

0. 前提条件：ライブラリのインポート、APIキーの設定（必要な場合）
1. 収集：2022年オリンピックに関する数百のWikipedia記事をダウンロードします
2. チャンク：ドキュメントは短い、半自己完結のセクションに分割され、埋め込み対象となります
3. 埋め込み：各セクションはOpenAI APIで埋め込まれます
4. 保存：埋め込みデータはCSVファイルに保存されます（大規模なデータセットの場合、ベクトルデータベースを使用します）

## 0. 前提条件

### ライブラリのインポート


In [1]:
# インポート
import mwclient  # サンプルのWikipedia記事をダウンロードするため
import mwparserfromhell  # Wikipedia記事をセクションに分割するため
import openai  # 埋め込みを生成するため
import pandas as pd  # 記事セクションと埋め込みを格納するためのDataFrame
import re  # Wikipedia記事から<ref>リンクを取り除くため
import tiktoken  # トークンを数えるため


ターミナルで `pip install` を使用して不足しているライブラリをインストールしてください。例：

```zsh
pip install openai
```

（ノートブックセルでも `!pip install openai` を使用できます。）

ライブラリをインストールした場合は、ノートブックカーネルを再起動してください。

### APIキーの設定（必要な場合）

OpenAIライブラリは、`OPENAI_API_KEY`環境変数からAPIキーを読み込もうとします。まだ設定していない場合は、[こちらの手順](https://help.openai.com/en/articles/5112595-best-practices-for-api-key-safety)に従ってこの環境変数を設定してください。

## 1. ドキュメントを収集する

この例では、2022年冬季オリンピックに関連する数百のウィキペディア記事をダウンロードします。

In [2]:
# 2022年冬季オリンピックに関するWikipediaページを取得する

CATEGORY_TITLE = "Category:2022 Winter Olympics"
WIKI_SITE = "en.wikipedia.org"


def titles_from_category(
    category: mwclient.listing.Category, max_depth: int
) -> set[str]:
    """指定したWikiカテゴリとそのサブカテゴリ内のページタイトルのセットを返す関数。"""
    titles = set()
    for cm in category.members():
        if type(cm) == mwclient.page.Page:
            # isinstance()の代わりにtype()を使用して、継承のないマッチをキャッチします
            titles.add(cm.name)
        elif isinstance(cm, mwclient.listing.Category) and max_depth > 0:
            deeper_titles = titles_from_category(cm, max_depth=max_depth - 1)
            titles.update(deeper_titles)
    return titles


site = mwclient.Site(WIKI_SITE)
category_page = site.pages[CATEGORY_TITLE]
titles = titles_from_category(category_page, max_depth=1)
# max_depth=1は、カテゴリツリーを1レベル深く探索することを意味します
print(f"{CATEGORY_TITLE}内の記事タイトルを {len(titles)} 件見つけました。")




Category:2022 Winter Olympics内の記事タイトルを 732 件見つけました。


## 2. ドキュメントをチャンク化する

参照ドキュメントが用意できたので、それを検索のために準備する必要があります。

GPTは一度に読むことができるテキスト量に制限があるため、各ドキュメントを読むのに十分に短いチャンクに分割します。

この具体的な例では、Wikipediaの記事に関して以下の作業を行います：
- 外部リンクや脚注など、あまり関連性の低いセクションは削除します。
- 参照タグ（例：<ref>）や空白、非常に短いセクションを削除してテキストをクリーンアップします。
- 各記事をセクションに分割します。
- 各セクションのテキストの前にタイトルとサブタイトルを追加し、GPTがコンテキストを理解するのを支援します。
- セクションが長い場合（例：1,600トークン以上）、段落などの意味的な境界に沿って再帰的に小さなセクションに分割しようとします。

In [5]:
from typing import List, Tuple, Set
import mwparserfromhell
import mwclient


SECTIONS_TO_IGNORE = [
    "See also",
    "References",
    "External links",
    "Further reading",
    "Footnotes",
    "Bibliography",
    "Sources",
    "Citations",
    "Literature",
    "Footnotes",
    "Notes and references",
    "Photo gallery",
    "Works cited",
    "Photos",
    "Gallery",
    "Notes",
    "References and sources",
    "References and notes",
]


def all_subsections_from_section(
    section: mwparserfromhell.wikicode.Wikicode,
    parent_titles: list[str],
    sections_to_ignore: set[str],
) -> list[tuple[list[str], str]]:
    """
    Wikipediaのセクションから、すべてのネストされたセクションをフラット化したリストを返す関数。
    各セクションはタプルであり、次のように構成されます：
        - 最初の要素は親のサブタイトルのリストで、ページタイトルから始まります
        - 2番目の要素はセクションのテキストです（子セクションは含まれません）
    """
    headings = [str(h) for h in section.filter_headings()]
    title = headings[0]
    if title.strip("=" + " ") in sections_to_ignore:
        # ^ウィキの見出しは "== 見出し ==" のようにラップされています
        return []
    titles = parent_titles + [title]
    full_text = str(section)
    section_text = full_text.split(title)[1]
    if len(headings) == 1:
        return [(titles, section_text)]
    else:
        first_subtitle = headings[1]
        section_text = section_text.split(first_subtitle)[0]
        results = [(titles, section_text)]
        for subsection in section.get_sections(levels=[len(titles) + 1]):
            results.extend(all_subsections_from_section(subsection, titles, sections_to_ignore))
        return results

def all_subsections_from_title(
    title: str,
    sections_to_ignore: set[str] = SECTIONS_TO_IGNORE,
    site_name: str = WIKI_SITE,
) -> list[tuple[list[str], str]]:
    """Wikipediaのページタイトルから、すべてのネストされたセクションをフラット化したリストを返す関数。
    各セクションはタプルであり、次のように構成されます：
        - 最初の要素は親のサブタイトルのリストで、ページタイトルから始まります
        - 2番目の要素はセクションのテキストです（子セクションは含まれません）
    """
    site = mwclient.Site(site_name)
    page = site.pages[title]
    text = page.text()
    parsed_text = mwparserfromhell.parse(text)
    headings = [str(h) for h in parsed_text.filter_headings()]
    if headings:
        summary_text = str(parsed_text).split(headings[0])[0]
    else:
        summary_text = str(parsed_text)
    results = [([title], summary_text)]
    for subsection in parsed_text.get_sections(levels=[2]):
        results.extend(all_subsections_from_section(subsection, [title], sections_to_ignore))
    return results

In [7]:
from tqdm.auto import tqdm
# ページをセクションに分割
# 100記事あたり約1分かかる場合があります
wikipedia_sections = []
for title in tqdm(titles):
    wikipedia_sections.extend(all_subsections_from_title(title))
print(f"{len(titles)} ページで {len(wikipedia_sections)} セクションが見つかりました。")


  0%|          | 0/732 [00:00<?, ?it/s]

732 ページで 5744 セクションが見つかりました。


In [5]:
# テキストをクリーンアップする
def clean_section(section: tuple[list[str], str]) -> tuple[list[str], str]:
    """
    <ref>xyz</ref> パターンを削除し、先頭および末尾の空白を削除してクリーンなセクションを返します。
    """
    titles, text = section
    text = re.sub(r"<ref.*?</ref>", "", text)
    text = text.strip()
    return (titles, text)

# 短い/空のセクションをフィルタリングする
def keep_section(section: tuple[list[str], str]) -> bool:
    """セクションを保持すべき場合はTrueを返し、それ以外の場合はFalseを返します。"""
    titles, text = section
    if len(text) < 16:
        return False
    else:
        return True

original_num_sections = len(wikipedia_sections)
wikipedia_sections = [ws for ws in wikipedia_sections if keep_section(ws)]
print(f"{original_num_sections-len(wikipedia_sections)} 個のセクションをフィルタリングし、{len(wikipedia_sections)} 個のセクションが残りました。")


Filtered out 530 sections, leaving 5200 sections.


In [6]:
# print example data
for ws in wikipedia_sections[:5]:
    print(ws[0])
    display(ws[1][:77] + "...")
    print()


['Lviv bid for the 2022 Winter Olympics']


'{{Olympic bid|2022|Winter|\n| Paralympics = yes\n| logo = Lviv 2022 Winter Olym...'


['Lviv bid for the 2022 Winter Olympics', '==History==']


'[[Image:Lwów - Rynek 01.JPG|thumb|right|200px|View of Rynok Square in Lviv]]\n...'


['Lviv bid for the 2022 Winter Olympics', '==Venues==']


'{{Location map+\n|Ukraine\n|border =\n|caption = Venue areas\n|float = left\n|widt...'


['Lviv bid for the 2022 Winter Olympics', '==Venues==', '===City zone===']


'The main Olympic Park would be centered around the [[Arena Lviv]], hosting th...'


['Lviv bid for the 2022 Winter Olympics', '==Venues==', '===Mountain zone===', '====Venue cluster Tysovets-Panasivka====']


'An existing military ski training facility in [[Tysovets, Skole Raion|Tysovet...'

次に、長いセクションを再帰的により小さなセクションに分割します。

テキストをセクションに分割するための完璧なレシピは存在しません。

いくつかのトレードオフがあります：
- より長いセクションは、より多くの文脈が必要な質問には向いているかもしれません
- より長いセクションは、回収には不利かもしれません。なぜなら、複数のトピックが混在しているかもしれないからです
- より短いセクションはコストを削減するために良いです（コストはトークンの数に比例しています）
- より短いセクションは、より多くのセクションを回収することができ、これはリコールに役立つかもしれません
- オーバーラップするセクションは、セクションの境界で回答が切れるのを防ぐのに役立つかもしれません

ここでは、シンプルなアプローチを使用し、セクションを最大1,600トークンに制限し、長すぎるセクションを再帰的に半分に分割します。有用な文の途中で切断されるのを防ぐために、可能な限り段落の境界で分割します。

In [7]:
from typing import List, Tuple, Union

GPT_MODEL = "gpt-3.5-turbo"  # トークナイザーを選択するためだけに重要

def num_tokens(text: str, model: str = GPT_MODEL) -> int:
    """
    文字列内のトークン数を返します。
    """
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

def halved_by_delimiter(string: str, delimiter: str = "\n") -> List[str]:
    """
    デリミタによって文字列を二つに分割し、各サイドのトークンをバランスよく保ちます。
    """
    chunks = string.split(delimiter)
    if len(chunks) == 1:
        return [string, ""]  # デリミタが見つからない
    elif len(chunks) == 2:
        return chunks  # 中間点を探す必要はない
    else:
        total_tokens = num_tokens(string)
        halfway = total_tokens // 2
        best_diff = halfway
        for i, _ in enumerate(chunks):
            left = delimiter.join(chunks[: i + 1])
            left_tokens = num_tokens(left)
            diff = abs(halfway - left_tokens)
            if diff >= best_diff:
                break
            else:
                best_diff = diff
        left = delimiter.join(chunks[:i])
        right = delimiter.join(chunks[i:])
        return [left, right]

def truncated_string(
    string: str, model: str, max_tokens: int, print_warning: bool = True
) -> str:
    """
    文字列を最大トークン数に制限して切り詰めます。
    """
    encoding = tiktoken.encoding_for_model(model)
    encoded_string = encoding.encode(string)
    truncated_result = encoding.decode(encoded_string[:max_tokens])
    if print_warning and len(encoded_string) > max_tokens:
        print(f"Warning: Truncated string from {len(encoded_string)} tokens to {max_tokens} tokens.")
    return truncated_result

def split_strings_from_subsection(
    subsection: Tuple[List[str], str],
    max_tokens: int = 1000,
    model: str = GPT_MODEL,
    max_recursion: int = 5,
) -> List[str]:
    """
    サブセクションを最大トークン数に分割し、リストとして返します。
    各サブセクションは親タイトル [H1, H2, ...] とテキスト (str) のタプルです。
    """
    titles, text = subsection
    full_string = "\n\n".join(titles + [text])
    num_tokens_in_string = num_tokens(full_string)
    
    if num_tokens_in_string <= max_tokens:
        return [full_string]
    elif max_recursion == 0:
        return [truncated_string(full_string, model=model, max_tokens=max_tokens)]
    else:
        for delimiter in ["\n\n", "\n", ". "]:
            left, right = halved_by_delimiter(text, delimiter=delimiter)
            if left == "" or right == "":
                continue
            else:
                results = []
                for half in [left, right]:
                    half_subsection = (titles, half)
                    half_strings = split_strings_from_subsection(
                        half_subsection,
                        max_tokens=max_tokens,
                        model=model,
                        max_recursion=max_recursion - 1,
                    )
                    results.extend(half_strings)
                return results
    return [truncated_string(full_string, model=model, max_tokens=max_tokens)]


In [8]:
# セクションをチャンクに分割
MAX_TOKENS = 1600
wikipedia_strings = []
for section in wikipedia_sections:
    wikipedia_strings.extend(split_strings_from_subsection(section, max_tokens=MAX_TOKENS))

print(f"{len(wikipedia_sections)}のWikipediaセクションを{len(wikipedia_strings)}の文字列に分割しました。")


5200 Wikipedia sections split into 6059 strings.


In [9]:
# print example data
print(wikipedia_strings[1])


Lviv bid for the 2022 Winter Olympics

==History==

[[Image:Lwów - Rynek 01.JPG|thumb|right|200px|View of Rynok Square in Lviv]]

On 27 May 2010, [[President of Ukraine]] [[Viktor Yanukovych]] stated during a visit to [[Lviv]] that Ukraine "will start working on the official nomination of our country as the holder of the Winter Olympic Games in [[Carpathian Mountains|Carpathians]]".

In September 2012, [[government of Ukraine]] approved a document about the technical-economic substantiation of the national project "Olympic Hope 2022". This was announced by Vladyslav Kaskiv, the head of Ukraine´s Derzhinvestproekt (State investment project). The organizers announced on their website venue plans featuring Lviv as the host city and location for the "ice sport" venues, [[Volovets]] (around {{convert|185|km|mi|abbr=on}} from Lviv) as venue for the [[Alpine skiing]] competitions and [[Tysovets, Skole Raion|Tysovets]] (around {{convert|130|km|mi|abbr=on}} from Lviv) as venue for all other "sn

## 3. ドキュメントチャンクの埋め込み

ライブラリを短い自己完結型の文字列に分割したので、各文字列の埋め込みを計算できます。

（大規模な埋め込みのジョブについては、[api_request_parallel_processor.py](api_request_parallel_processor.py)のようなスクリプトを使用して、レート制限を守りながらリクエストを並列化することができます。）

In [10]:
# 埋め込みを計算します
EMBEDDING_MODEL = "text-embedding-ada-002"  # 2023年4月現在、OpenAIの最高の埋め込みモデル
BATCH_SIZE = 1000  # 1リクエストあたり最大2048個の埋め込み入力を提供できます

embeddings = []
for batch_start in range(0, len(wikipedia_strings), BATCH_SIZE):
    batch_end = batch_start + BATCH_SIZE
    batch = wikipedia_strings[batch_start:batch_end]
    print(f"バッチ {batch_start} から {batch_end-1} の計算中")
    response = openai.Embedding.create(model=EMBEDDING_MODEL, input=batch)
    for i, be in enumerate(response["data"]):
        assert i == be["index"]  # 埋め込みが入力と同じ順序であることをダブルチェック
    batch_embeddings = [e["embedding"] for e in response["data"]]
    embeddings.extend(batch_embeddings)

df = pd.DataFrame({"text": wikipedia_strings, "embedding": embeddings})


Batch 0 to 999
Batch 1000 to 1999
Batch 2000 to 2999
Batch 3000 to 3999
Batch 4000 to 4999
Batch 5000 to 5999
Batch 6000 to 6999


## 4. ドキュメントの断片と埋め込みを保存

この例では、数千の文字列しか使用しないため、それらをCSVファイルに保存します。

（より大きなデータセットの場合、パフォーマンスが向上するベクトルデータベースを使用してください。）

In [11]:
# save document chunks and embeddings

SAVE_PATH = "data/winter_olympics_2022.csv"

df.to_csv(SAVE_PATH, index=False)
